In [9]:
import pandas as pd
# 데이터 불러오기
file_path_train = '/root/base_data/test_2_modified2_updated.csv'
train_df = pd.read_csv(file_path_train, encoding='utf-8')


final_coords_df=pd.read_csv('/root/base_data/최종위도경도.csv', encoding='utf-8')

/tmp/ipykernel_505709/4229198051.py:4: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(file_path_train, encoding='utf-8')


In [10]:
train_df.shape

(9272, 82)

In [8]:
df.shape

(9272, 51)

In [11]:
train_df.head()

,city_district,lot_number,main_number,sub_number,apartment_name,exclusive_area,contract_year_month,contract_day,floor,construction_year,...,distance_middle,distance_elem50,distance_elem75,distance_elem100,distance_middle50,distance_middle75,distance_middle100,daechi_mokdong,contract_month,high_achivement
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,595.817686,0,0,1,0,1,0,0,0,0
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,111.901395,1,0,0,1,0,0,0,0,1
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,225.953458,1,0,0,1,0,0,0,0,1
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,225.953458,1,0,0,1,0,0,0,0,1
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,225.953458,1,0,0,1,0,0,0,0,1


In [12]:
# 결측치가 있는 행 추출
missing_values_df =train_df[train_df[['coordinate_x', 'coordinate_y']].isnull().any(axis=1)]

# 결측치가 있는 행들 출력
print("결측치가 있는 행들:")
print(missing_values_df)

# 결측치가 있는 행 수 출력
print(f"결측치가 있는 행의 수: {missing_values_df.shape[0]}")

결측치가 있는 행들:
      city_district lot_number  main_number  sub_number apartment_name  \
113   서울특별시 강남구 개포동        189        189.0         0.0      개포자이프레지던스   
114   서울특별시 강남구 개포동        185        185.0         0.0        개포주공7단지   
115   서울특별시 강남구 개포동        185        185.0         0.0        개포주공7단지   
116   서울특별시 강남구 개포동        185        185.0         0.0        개포주공7단지   
117   서울특별시 강남구 개포동        185        185.0         0.0        개포주공7단지   
...             ...        ...          ...         ...            ...   
9253  서울특별시 중랑구 중화동        450        450.0         0.0             한신   
9254  서울특별시 중랑구 중화동        450        450.0         0.0             한신   
9255  서울특별시 중랑구 중화동        450        450.0         0.0             한신   
9256  서울특별시 중랑구 중화동        450        450.0         0.0             한신   
9257  서울특별시 중랑구 중화동        450        450.0         0.0             한신   

      exclusive_area  contract_year_month  contract_day  floor  \
113            59.98             

In [13]:
train_df.columns

Index(['city_district', 'lot_number', 'main_number', 'sub_number',
       'apartment_name', 'exclusive_area', 'contract_year_month',
       'contract_day', 'floor', 'construction_year', 'road_name',
       'cancellation_reason_date', 'registration_application_date',
       'transaction_type', 'broker_location', 'complex_classification',
       'phone_number', 'fax_number', 'complex_description', 'household_type',
       'management_type', 'corridor_type', 'heating_type', 'total_buildings',
       'total_households', 'construction_company', 'developer',
       'approval_date', 'gross_area', 'residential_area', 'maintenance_area',
       'household_status_below_60sqm', 'household_status_60_to_85sqm',
       'household_status_85_to_135sqm', 'household_status_above_135sqm',
       'website', 'registration_date', 'modification_date',
       'employment_insurance_number', 'security_management_type',
       'electricity_contract_type', 'cleaning_management_type',
       'building_area', 'park

In [14]:
# train_df에서 기존 'unique_key' 컬럼 삭제
train_df = train_df.drop(columns=['unique_key'], errors='ignore')

# 새로운 'unique_key' 생성: 'city_district' + ' ' + 'lot_number'
train_df['unique_key'] = train_df['city_district'] + ' ' + train_df['lot_number'].fillna('nan')


In [15]:
train_df['unique_key'].isnull().sum()

0

In [16]:
train_df.shape

(9272, 82)

In [33]:
'''import pandas as pd
from fuzzywuzzy import fuzz, process
from tqdm import tqdm

# 예제 데이터프레임 (train_df와 final_coords_df가 있다고 가정)
# train_df = pd.read_csv('your_train_data.csv')
# final_coords_df = pd.read_csv('your_coords_data.csv')

# 정확히 일치하는 항목을 먼저 병합
merged_exact_df = pd.merge(
    train_df,
    final_coords_df[['address', 'Longitude', 'Latitude']],
    left_on='unique_key',
    right_on='address',
    how='left'
)

# 유사 문자열 매칭을 위한 함수
def merge_fuzzy(df_1, df_2, key1, key2, threshold=95):
    matched_rows = []
    for index, row in tqdm(df_1.iterrows(), total=df_1.shape[0], desc="Processing rows"):
        match = process.extractOne(row[key1], df_2[key2], scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold:
            matched_row = df_2[df_2[key2] == match[0]]
            matched_row = matched_row.assign(**row)
            matched_rows.append(matched_row)
    return pd.concat(matched_rows, ignore_index=True) if matched_rows else pd.DataFrame()

# 정확히 일치하지 않은 항목들에 대해 유사도 매칭 수행
unmatched = merged_exact_df[merged_exact_df['Longitude'].isnull() & merged_exact_df['Latitude'].isnull()]

# unmatched 데이터 확인
print("Unmatched data:")
print(unmatched[['unique_key', 'address']].head())  # unique_key와 address 컬럼만 확인

# 유사도 매칭을 통한 병합
fuzzy_matched_df = merge_fuzzy(unmatched, final_coords_df, 'unique_key', 'address')

# 매칭된 데이터 확인
print("Fuzzy matched data:")
print(fuzzy_matched_df[['unique_key', 'address', 'Longitude', 'Latitude']].head())  # unique_key, address, Longitude, Latitude 컬럼 확인

# 최종 병합 결과
merged_fuzzy_df = pd.concat([merged_exact_df[~merged_exact_df.index.isin(unmatched.index)], fuzzy_matched_df])

# 유사도 매칭 과정에서 누락된 데이터 확인
print(f"Initial unmatched count: {unmatched.shape[0]}")
print(f"Fuzzy matched count: {fuzzy_matched_df.shape[0]}")
print(f"Remaining unmatched count after fuzzy matching: {unmatched.shape[0] - fuzzy_matched_df.shape[0]}")

# 결과 출력
print("Merged fuzzy df head:")
print(merged_fuzzy_df.head())

# 결측치가 있는 행 추출
missing_values_df = merged_fuzzy_df[merged_fuzzy_df[['Longitude', 'Latitude']].isnull().any(axis=1)]

# 결측치가 있는 행들 출력
print("결측치가 있는 행들:")
print(missing_values_df)

# 결측치가 있는 행 수 출력
print(f"결측치가 있는 행의 수: {missing_values_df.shape[0]}")
'''

Unmatched data:
                unique_key address
458   서울특별시 강남구 삼성동 142-16     NaN
585    서울특별시 강남구 역삼동 712-5     NaN
1173     서울특별시 강동구 천호동 578     NaN
1174     서울특별시 강동구 천호동 578     NaN
1274    서울특별시 강북구 번동 443-6     NaN


Processing rows:   0%|          | 0/73 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 73/73 [00:04<00:00, 16.12it/s]


Fuzzy matched data:


KeyError: "None of [Index(['unique_key', 'address', 'Longitude', 'Latitude'], dtype='object')] are in the [columns]"

In [47]:
import pandas as pd

# 예제 데이터프레임 (train_df와 final_coords_df가 있다고 가정)
# train_df = pd.read_csv('your_train_data.csv')
# final_coords_df = pd.read_csv('your_coords_data.csv')

# 정확히 일치하는 항목을 병합
merged_exact_df = pd.merge(
    train_df,
    final_coords_df[['address', 'Longitude', 'Latitude']],
    left_on='unique_key',
    right_on='address',
    how='left'
)

# 병합 결과에서 Longitude를 coordinate_x로, Latitude를 coordinate_y로 변경
merged_exact_df['coordinate_x'] = merged_exact_df['Longitude']
merged_exact_df['coordinate_y'] = merged_exact_df['Latitude']

# 필요 없는 컬럼 삭제
merged_exact_df.drop(columns=['Longitude', 'Latitude', 'address'], inplace=True)

# 병합 결과 확인
print("Merged exact data frame:")
print(merged_exact_df.head())

print(merged_exact_df['coordinate_x'].isnull().sum())

Merged exact data frame:
   city_district lot_number  main_number  sub_number apartment_name  \
0  서울특별시 강남구 개포동      658-1        658.0         1.0         개포6차우성   
1  서울특별시 강남구 개포동      651-1        651.0         1.0        개포더샵트리에   
2  서울특별시 강남구 개포동        652        652.0         0.0         개포우성3차   
3  서울특별시 강남구 개포동        652        652.0         0.0         개포우성3차   
4  서울특별시 강남구 개포동        652        652.0         0.0         개포우성3차   

   exclusive_area  contract_year_month  contract_day  floor  \
0         79.9700               202307            26      5   
1        108.2017               202308            15     10   
2        161.0000               202307            28     15   
3        133.4600               202308            10     14   
4        104.4300               202308            18      6   

   construction_year  ... distance_elem50  distance_elem75 distance_elem100  \
0               1987  ...               0                0                1   
1          

In [49]:
# 결측치가 있는 행 추출
missing_values_df = merged_exact_df[merged_exact_df[['coordinate_x', 'coordinate_y']].isnull().any(axis=1)]

# 결측치가 있는 행들 출력
print("Rows with missing values:")
print(missing_values_df)

# 결측치가 있는 행 수 출력
print(f"Number of rows with missing values: {missing_values_df.shape[0]}")

print(missing_values_df['lot_number'].isnull().sum())

Rows with missing values:
       city_district lot_number  main_number  sub_number apartment_name  \
458    서울특별시 강남구 삼성동     142-16        142.0        16.0           한양립스   
585    서울특별시 강남구 역삼동      712-5        712.0         5.0    역삼센트럴2차아이파크   
1173   서울특별시 강동구 천호동        578        578.0         0.0        힐데스하임천호   
1174   서울특별시 강동구 천호동        578        578.0         0.0        힐데스하임천호   
1274    서울특별시 강북구 번동      443-6        443.0         6.0       아츠스테이수유점   
...              ...        ...          ...         ...            ...   
4738   서울특별시 마포구 신수동      400-1        400.0         1.0        신수동르끌레브   
6087   서울특별시 성동구 송정동       67-5         67.0         5.0      히페리온리버팰리스   
8880  서울특별시 중구 인현동2가        240        240.0         0.0     세운푸르지오헤리시티   
8881  서울특별시 중구 인현동2가        240        240.0         0.0     세운푸르지오헤리시티   
9050    서울특별시 중구 입정동        277        277.0         0.0  힐스테이트세운센트럴1단지   

      exclusive_area  contract_year_month  contract_day  floor  \
458    

In [ ]:
from geopy.geocoders import Nominatim
import time
# Geopy를 사용하여 결측치 채우기
geolocator = Nominatim(user_agent="geoapiExercises")

def get_lat_lon(row):
    address = f"{row['city_district']} {row['lot_number']}"
    try:
        location = geolocator.geocode(address)
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    return pd.Series([None, None])

# 결측치가 있는 행에 대해 위도 및 경도 얻어내기
missing_values_df[['coordinate_y', 'coordinate_x']] = missing_values_df.apply(get_lat_lon, axis=1)

# 원래 데이터프레임에 결측치 채운 값 병합
merged_exact_df.update(missing_values_df)

# 결과 출력
print("Updated data frame with filled missing values:")
print(merged_exact_df)

# 결측치가 있는 행 수 다시 출력
missing_values_df = merged_exact_df[merged_exact_df[['coordinate_x', 'coordinate_y']].isnull().any(axis=1)]
print(f"Number of rows with missing values after geocoding: {missing_values_df.shape[0]}")

In [56]:
merged_exact_df.shape

(9272, 82)

In [55]:
merged_exact_df.to_csv('/root/code/test_2_modified2_updated(test_shape해결_결측치0).csv')

In [34]:
'''import pandas as pd
from fuzzywuzzy import fuzz, process
from tqdm import tqdm

# 정확히 일치하는 항목을 먼저 병합
merged_exact_df = pd.merge(
    train_df,
    final_coords_df[['address', 'Longitude', 'Latitude']],
    left_on='unique_key',
    right_on='address',
    how='left'
)

# 유사 문자열 매칭을 위한 함수
def merge_fuzzy(df_1, df_2, key1, key2, threshold=95):
    matched_rows = []
    for index, row in tqdm(df_1.iterrows(), total=df_1.shape[0], desc="Processing rows"):
        match = process.extractOne(row[key1], df_2[key2], scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold:
            matched_row = df_2[df_2[key2] == match[0]]
            matched_row = matched_row.assign(**row)
            matched_rows.append(matched_row)
    return pd.concat(matched_rows, ignore_index=True) if matched_rows else pd.DataFrame()

# 정확히 일치하지 않은 항목들에 대해 유사도 매칭 수행
unmatched = merged_exact_df[merged_exact_df['Longitude'].isnull() & merged_exact_df['Latitude'].isnull()]
# 유사도 매칭을 통한 병합
fuzzy_matched_df = merge_fuzzy(unmatched, final_coords_df, 'unique_key', 'address')
print(fuzzy_matched_df)
# 최종 병합 결과
merged_fuzzy_df = pd.concat([merged_exact_df[~merged_exact_df.index.isin(unmatched.index)], fuzzy_matched_df])
'''


Processing rows:   0%|          | 0/73 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 73/73 [00:04<00:00, 16.01it/s]


Empty DataFrame
Columns: []
Index: []
city_district       0
lot_number          2
main_number         0
sub_number          0
apartment_name     10
                   ..
high_achivement     0
unique_key          0
address             0
Longitude           0
Latitude            0
Length: 85, dtype: int64


In [39]:
print(merged_fuzzy_df['coordinate_x'].isnull().sum())
print(merged_fuzzy_df.shape)

6491
(9199, 85)


In [32]:
# 결과 출력
print(merged_fuzzy_df['coordinate_x'].isnull().sum())

# 결측치가 있는 행 추출
missing_values_df = merged_fuzzy_df[merged_fuzzy_df[['Longitude', 'Latitude']].isnull().any(axis=1)]

6491


In [18]:
merged_fuzzy_df.shape

(9199, 85)

In [132]:
merged_fuzzy_df.isnull().sum()

city_district        0
lot_number           2
main_number          0
sub_number           0
apartment_name      10
                    ..
unique_key           0
unique_key_clean     0
address              0
Longitude            0
Latitude             0
Length: 86, dtype: int64

In [133]:
merged_fuzzy_df.to_csv('/root/code/test_2_modified2_updated(위도_경도_결측치0).csv', index=False)

In [134]:
merged_fuzzy_df.columns

Index(['city_district', 'lot_number', 'main_number', 'sub_number',
       'apartment_name', 'exclusive_area', 'contract_year_month',
       'contract_day', 'floor', 'construction_year', 'road_name',
       'cancellation_reason_date', 'registration_application_date',
       'transaction_type', 'broker_location', 'complex_classification',
       'phone_number', 'fax_number', 'complex_description', 'household_type',
       'management_type', 'corridor_type', 'heating_type', 'total_buildings',
       'total_households', 'construction_company', 'developer',
       'approval_date', 'gross_area', 'residential_area', 'maintenance_area',
       'household_status_below_60sqm', 'household_status_60_to_85sqm',
       'household_status_85_to_135sqm', 'household_status_above_135sqm',
       'website', 'registration_date', 'modification_date',
       'employment_insurance_number', 'security_management_type',
       'electricity_contract_type', 'cleaning_management_type',
       'building_area', 'park

In [135]:
merged_fuzzy_df['within_250m'].isnull().sum()

0

In [114]:
# 결측치가 있는 행 추출
missing_values_df = merged_df[merged_df[['Longitude', 'Latitude']].isnull().any(axis=1)]

# 결측치가 있는 행들 출력
print("결측치가 있는 행들:")
print(missing_values_df)

# 결측치가 있는 행 수 출력
print(f"결측치가 있는 행의 수: {missing_values_df.shape[0]}")

결측치가 있는 행들:
       city_district lot_number  main_number  sub_number apartment_name  \
458    서울특별시 강남구 삼성동     142-16        142.0        16.0           한양립스   
585    서울특별시 강남구 역삼동      712-5        712.0         5.0    역삼센트럴2차아이파크   
1173   서울특별시 강동구 천호동        578        578.0         0.0        힐데스하임천호   
1174   서울특별시 강동구 천호동        578        578.0         0.0        힐데스하임천호   
1274    서울특별시 강북구 번동      443-6        443.0         6.0       아츠스테이수유점   
...              ...        ...          ...         ...            ...   
4738   서울특별시 마포구 신수동      400-1        400.0         1.0        신수동르끌레브   
6087   서울특별시 성동구 송정동       67-5         67.0         5.0      히페리온리버팰리스   
8880  서울특별시 중구 인현동2가        240        240.0         0.0     세운푸르지오헤리시티   
8881  서울특별시 중구 인현동2가        240        240.0         0.0     세운푸르지오헤리시티   
9050    서울특별시 중구 입정동        277        277.0         0.0  힐스테이트세운센트럴1단지   

      exclusive_area  contract_year_month  contract_day  floor  \
458          38.2471 

In [120]:
# 'address' 열에 '서울특별시 중랑구 중화동'이 포함된 행들 찾기
filtered_data = final_coords_df[final_coords_df['address'].str.contains('강남구 삼성동 142', na=False)]

# 필터링된 데이터 출력
print("서울특별시 중랑구 중화동이 포함된 address 행들:")
print(filtered_data)

서울특별시 중랑구 중화동이 포함된 address 행들:
Empty DataFrame
Columns: [address, Latitude, Longitude, address_clean]
Index: []


In [89]:
# 병합된 데이터프레임을 새로운 CSV 파일로 저장
merged_df.to_csv('/root/code/train_2_modified2_updated(위도_경도_결측치0).csv', index=False)


In [73]:
indices_to_check = [720188, 720189]
specific_rows = merged_df.loc[indices_to_check]

specific_rows

,city_district,lot_number,main_number,sub_number,apartment_name,exclusive_area,contract_year_month,contract_day,floor,construction_year,...,distance_elem100,distance_middle50,distance_middle75,distance_middle100,daechi_mokdong,contract_month,high_achivement,cleaned_address,coordinate_x,coordinate_y
720188,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201504,17,9,2014,...,1,0,0,1,0,0,0,NaN,NaN,NaN
720189,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201505,1,6,2014,...,1,0,0,1,0,0,0,NaN,NaN,NaN


In [29]:
train['unique_key']

0          서울특별시 강남구 개포동 658-1
1          서울특별시 강남구 개포동 658-1
2          서울특별시 강남구 개포동 658-1
3          서울특별시 강남구 개포동 658-1
4          서울특별시 강남구 개포동 658-1
                  ...         
1118817      서울특별시 은평구 구산동 382
1118818      서울특별시 은평구 구산동 382
1118819      서울특별시 은평구 구산동 382
1118820      서울특별시 은평구 구산동 382
1118821     서울특별시 중구 묵정동 11-67
Name: unique_key, Length: 1118822, dtype: object

In [22]:
train[['coordinate_x', 'coordinate_y', 'complex_application_date',
       'rest_area_food',  'cinema_distance',
       'large_store_distance', 'government_bond_3yr',  'within_250m', 'within_500m', 'beyond_500m',
       'nearest_elementary', 'nearest_middle', 'distance_elementary',
       'distance_middle', 'distance_elem50', 'distance_elem75',
       'distance_elem100', 'distance_middle50', 'distance_middle75',
       'distance_middle100', 'daechi_mokdong', 'contract_month',
       'high_achivement' ]].isnull().sum()

coordinate_x                869670
coordinate_y                869670
complex_application_date    869625
rest_area_food               52572
cinema_distance             869670
large_store_distance        869670
government_bond_3yr              0
within_250m                      0
within_500m                      0
beyond_500m                      0
nearest_elementary               0
nearest_middle                   0
distance_elementary              0
distance_middle                  0
distance_elem50                  0
distance_elem75                  0
distance_elem100                 0
distance_middle50                0
distance_middle75                0
distance_middle100               0
daechi_mokdong                   0
contract_month                   0
high_achivement                  0
dtype: int64

In [23]:
test[['coordinate_x', 'coordinate_y', 'complex_application_date',
       'rest_area_food',  'cinema_distance',
       'large_store_distance', 'government_bond_3yr',  'within_250m', 'within_500m', 'beyond_500m',
       'nearest_elementary', 'nearest_middle', 'distance_elementary',
       'distance_middle', 'distance_elem50', 'distance_elem75',
       'distance_elem100', 'distance_middle50', 'distance_middle75',
       'distance_middle100', 'daechi_mokdong', 'contract_month',
       'high_achivement' ]].isnull().sum()

coordinate_x                6562
coordinate_y                6562
complex_application_date    6562
rest_area_food               300
cinema_distance             6562
large_store_distance        6562
government_bond_3yr            0
within_250m                   73
within_500m                   73
beyond_500m                   73
nearest_elementary          6562
nearest_middle              6562
distance_elementary         6562
distance_middle             6562
distance_elem50                0
distance_elem75                0
distance_elem100               0
distance_middle50              0
distance_middle75              0
distance_middle100             0
daechi_mokdong                 0
contract_month                 0
high_achivement                0
dtype: int64

In [7]:
data['coordinate_x'].isna().sum()

546812

In [ ]:
# '시군구'와 '번지'를 합쳐서 새로운 열 생성
data['unique_key'] = data['시군구'] + ' ' + data['번지']

# 'unique_key'를 기준으로 중복 행 제거
unique_data = data.drop_duplicates(subset=['unique_key'])

In [ ]:
# 'city_district'와 'lot_number'를 결합하여 새로운 'unique_key' 생성
dt['unique_key'] = dt['city_district'] + ' ' + dt['lot_number']
dt_test['unique_key'] = dt_test['city_district'] + ' ' + dt_test['lot_number']

In [ ]:
# unique_data의 컬럼 이름 변경
unique_data.rename(columns={'500m_내_정류소_수': 'within_500m_bus'}, inplace=True)



In [ ]:
# 병합할 컬럼들이 존재하는지 확인
missing_columns = [col for col in columns_to_merge if col not in unique_data.columns]
if missing_columns:
    print(f"병합할 컬럼이 존재하지 않습니다: {missing_columns}")
else:
    # 데이터프레임 병합
    merged_data = pd.merge(dt_test, unique_data[columns_to_merge], on='unique_key', how='left')
    merged_data.to_csv('/root/0715_test_data.csv', index=False)

In [ ]:
# 필요한 컬럼만 선택하여 병합
columns_to_merge = ['unique_key', 'within_500m_bus', 'nearest_distance', 'within_250m', 'within_500m', 'beyond_500m']
merged_data = pd.merge(dt_test, unique_data[columns_to_merge], on='unique_key', how='left')